# Импорты

In [1]:
import os
import pandas as pd

# Импорт данных

Получим список существующих спарсенных категорий

In [34]:
path_catalogs = './../data/raw/catalogs'
files_catalogs = os.listdir(path_catalogs)
files_catalogs

['obuvmuzhskayakedy_i_krossovki.csv']

Спарсим таблицы

In [35]:
dfs_calatalog = []

for f in files_catalogs:
    df = pd.read_csv(f'{path_catalogs}/{f}')
    dfs_calatalog.append(df)

print(f'Спарсено каталогов: {len(dfs_calatalog)}')

Спарсено каталогов: 1


# Очистка данных

Посмотрим на одну из таблиц с данными спарсенного каталога. Каждый продукт имеет свои данные с отзывами.

In [36]:
dfs_calatalog[0].head()

,product_article,product_name,product_rating,count_reviews
0,276942833,Кроссовки SNEAKERS,Нет оценок,0
1,250224621,Кроссовки утепленные с мехом,"4,7",100
2,259646173,Кроссовки мужские зимние с мехом высокие,"4,6",100
3,252655283,кроссовки зимние,"4,7",100
4,250220847,Кроссовки утепленные с мехом,"4,7",100


## Пример очистки данных на одной категории товаров

Так как на данный момент спарсен только 1 каталог, то очистим пошагово его. В дальнейшем можно будет свести в цикл для автоматизации.

In [37]:
df = dfs_calatalog[0]

### Удаление пустых значений

Посмотрим на уникальные значения

In [38]:
print(df['product_rating'].unique())
print(df['count_reviews'].unique())

['Нет оценок' '4,7' '4,6' '4,3' '4,8' '4,5' '4,9' '4,4']
[  0 100  65  82  79  30  42  24  31  10  78  40  63  23  96   9  32  12]


Видим товары без оценок, удалим их.

In [39]:
df = df[df['product_rating'] != 'Нет оценок']
df = df[df['count_reviews'] != 0]
df.head()

,product_article,product_name,product_rating,count_reviews
1,250224621,Кроссовки утепленные с мехом,"4,7",100
2,259646173,Кроссовки мужские зимние с мехом высокие,"4,6",100
3,252655283,кроссовки зимние,"4,7",100
4,250220847,Кроссовки утепленные с мехом,"4,7",100
5,252655288,кроссовки зимние,"4,7",100


Убедимся, что удалили

In [40]:
print(df['product_rating'].unique())
print(df['count_reviews'].unique())

['4,7' '4,6' '4,3' '4,8' '4,5' '4,9' '4,4']
[100  65  82  79  30  42  24  31  10  78  40  63  23  96   9  32  12]


### Удаление дубликатов

Бывает такое, что артикулы у товаров разные, но отзывы одни. Так как они созданы на одной странице товара, просто есть вариации товара. Для того чтобы понять, что это как раз этот случай — посмотрим на отзывы, так как у таких товаров одинаковые отзывы.

Удалим дубликаты, посредством сравнения.

In [84]:
path_reviews = f'./../data/raw/reviews/{files_catalogs[0][:-4]}'
files_reviews = os.listdir(path_reviews)
print(files_reviews)
df_clean = df

for f_main in files_reviews:
    df_main = pd.read_csv(f'{path_reviews}/{f_main}')
    path_save_reviews = f'./../data/clean/reviews/{files_catalogs[0][:-4]}'
    os.makedirs(path_save_reviews, exist_ok=True)
    files_save_reviews = os.listdir(path_save_reviews)
    are_equal = False

    # Сравниваем с сохранёнными файлами
    for f_save in files_save_reviews:
        df_save = pd.read_csv(f'{path_reviews}/{f_save}')
        are_equal = df_main.equals(df_save)
        if are_equal:  # если df_save == df_main
            df_clean = df_clean[df_clean['product_article'] != int(f_main[:-4])]  # удалить текущее значение из таблицу
            print(f'ОШИБКА! Отзыв {f_main} уже сохранён! Размер df_clean = {df_clean.shape}')
            break
    
    # Сравниваем с несохранёнными файлами
    if are_equal == False:
        for f_equal in files_reviews:
            df_equal = pd.read_csv(f'{path_reviews}/{f_equal}')
            are_equal = df_main.equals(df_equal)
            if are_equal:  # если df_equal == df_main
                if f_main == f_equal:  # если это
                    df_main.to_csv(f'./../data/clean/reviews/{files_catalogs[0][:-4]}/{f_main}', index=False)
                    # print(f'Найден сравняемый файл, сохраняем его {f_main}')
                else:
                    files_reviews.remove(f_equal)
                    df_clean = df_clean[df_clean['product_article'] != int(f_equal[:-4])]  # удалить текущее значение из таблицу
                    print(f'ОШИБКА! Найдено совпадение у файла {f_main} и {f_equal}. Размер df_clean = {df_clean.shape}')
        df_main.to_csv(f'./../data/clean/reviews/{files_catalogs[0][:-4]}/{f_main}', index=False)
        print(f'УСПЕХ! Сохранён уникальный отзыв {f_main}. Размер df_clean = {df_clean.shape}')
        
df_clean.to_csv(f'./../data/clean/catalogs/{files_catalogs[0]}', index=False)

['117770325.csv', '139698183.csv', '151507501.csv', '152272886.csv', '162306347.csv', '173928321.csv', '174535740.csv', '181318672.csv', '181542776.csv', '185600107.csv', '188783806.csv', '195459626.csv', '196143846.csv', '198140910.csv', '199542402.csv', '201389378.csv', '203589701.csv', '20927763.csv', '210689888.csv', '211305205.csv', '211628009.csv', '211875011.csv', '211877606.csv', '211877607.csv', '212943299.csv', '214860098.csv', '218797175.csv', '222116336.csv', '226121043.csv', '226121121.csv', '226121136.csv', '226121155.csv', '227834857.csv', '228653361.csv', '228653363.csv', '228653911.csv', '228653913.csv', '235210388.csv', '240648941.csv', '240648943.csv', '240652777.csv', '241002861.csv', '243487597.csv', '244772824.csv', '245171328.csv', '245171336.csv', '245171337.csv', '246433796.csv', '246434037.csv', '246434109.csv', '246434110.csv', '246454214.csv', '250220847.csv', '250224621.csv', '252655283.csv', '252655284.csv', '252655288.csv', '256315510.csv', '256874830.csv